<a href="https://colab.research.google.com/github/vydiep/MLProject/blob/main/CNN_DataPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import librosa
import numpy as np
import json

In [ ]:
def read_files(directory):
  '''
  Get .wav files from directory. Assign appropriate labels to each file.

  Parameters:
    directory: The data path

  Returns:
    file_paths: A list with tuples of file paths and their corresponding labels

  '''
    file_paths = []

    for root, directories, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".wav"):
                path = os.path.join(root, filename)
                if filename.startswith("Beethoven"):
                  label = 1
                else:
                  label = 0
                file_paths.append((path, label))

    return file_paths

def segment_audio_files_3s(audio):
  '''
  Load the audio for first 45 seconds. Segment the 45 seconds into 15 pieces of 3 seconds each.

  Parameters:
    audio: The path of the audio

  Returns:
    audio_list: A list of tuples containing the audio segments, their sample rate, and their corresponding label
  '''
    audio_list = []

    for file, label in audio:
        audio, sr = librosa.load(file, sr = 44100, duration = 45)
    
        segment_length = sr * 3

        for i in range(15):
            start = i * segment_length
            end = start + segment_length
            segment = audio[start:end]
            audio_list.append((segment, sr, label))

    return audio_list

def segment_audio_files_45s(audio):
  '''
  Load the audio for the first 45 seconds

  Parameters:
    audio: The audio path

  Returns:
    audio_list: A list of tuples containing the audio segment, their sample rate, and their corresponding label
  '''

    audio_list = []

    for file, label in audio:
        audio, sr = librosa.load(file, sr = 44100, duration = 45)
        audio_list.append((audio, sr, label))

    return audio_list

def generate_mel_spect(audio_files, json_name):
  '''
  Created a JSON file that contained spectrograms and labels

  Parameters:
    audio_files: The path of the audio files
    json_name: The name for the JSON file

  Returns:
    spectrograms: A list of the mel spectrograms generated using audio and sample rate
    labels: A list of labels


  '''
    spectrograms = []
    labels = []

    for audio, sr, label in audio_files:
        mel_spect = librosa.feature.melspectrogram(y = audio, sr = sr)
        mel_spect_db = librosa.power_to_db(mel_spect, ref = np.max)

        spectrograms.append(mel_spect.tolist())
        labels.append(label)

    data = {"spectrograms": spectrograms,
            "labels": labels}

    with open(json_name, "w") as f:
        json.dump(data, f)
        
    return spectrograms, labels